# 自动微分

教程地址：https://zh-v2.d2l.ai/chapter_preliminaries/autograd.html

## 1. 简单的例子

### 1.1 张量 x 的梯度

张量 $x$ 的梯度可以存储在 $x$ 上。

要点：

- `x.grad`: 取 $x$ 的梯度
- `x.requires_grad_(True)`: 允许 tenser $x$ 存储自己的梯度
- `x.grad.zero_()`: 将 $x$ 的梯度置零

In [1]:
import torch

# 初始化张量 x (tenser x)
x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

In [2]:
# help(x.requires_grad_)
x.requires_grad

False

In [3]:
x.requires_grad_(True)  # 允许 tensor x 存储梯度
x.grad == None  # 梯度默认为 None

True

初始化带梯度的张量，下面是两个例子：

In [4]:
torch.tensor([1., 2., 3.], requires_grad=True)

tensor([1., 2., 3.], requires_grad=True)

In [5]:
torch.randn((2, 5), requires_grad=True)

tensor([[-0.4565,  1.2696,  0.0927, -0.3001,  0.2633],
        [-0.2927, -0.2221,  0.8616,  0.2330, -1.1366]], requires_grad=True)

### 1.2 损失函数 及 反向传播

我们约定：

- 将 **损失** 记为 $y$
- 设 **损失函数** 为：$y = 2 * x \cdot x$（注意是点乘）

计算 $y$ 关于 $x$ 每个分量的梯度，步骤如下：

1. 定义损失函数：`y = 2 * torch.dot(x, x)`
2. 计算 $y$ 关于 $x$ 的梯度，即反向传播：`y.backward()`
3. 获取更新后 $x$ 的梯度：`x.grad`

In [6]:
y = 2 * torch.dot(x, x)
y  # 注意 y 在这里是标量

tensor(28., grad_fn=<MulBackward0>)

In [7]:
y.backward()  # 用反向传播自动计算 y 关于 x 每个分量的梯度
x.grad

tensor([ 0.,  4.,  8., 12.])

函数 $y = 2x^{T}x$ 关于 $x$ 的梯度应为 $4x$，验证是否正确：

In [8]:
x.grad == 4 * x

tensor([True, True, True, True])

## 2. 当损失为向量时

梯度的“形状”：

- 当损失 $y$ 为 **标量** 时，梯度是 **向量**，且与 $x$ 维度相同
- 当损失 $y$ 为 **向量** 时，梯度是 **矩阵**

注意，当损失 $y$ 为向量时。我们的目的不是计算微分矩阵，而是单独计算批量中每个样本的偏导数之和。也就是说：

在反向传播代码里要多加一个 `sum()` 函数，写成 `y.sum().backward()`

In [9]:
x.grad.zero_()  # 将张量 x 的梯度置零
x, x.grad

(tensor([0., 1., 2., 3.], requires_grad=True), tensor([0., 0., 0., 0.]))

定义损失函数：$y = 2 * x \times x$（注意是叉乘）

In [10]:
# 定义损失函数
y = 2 * x * x
y  # 注意 y 在这里是向量

tensor([ 0.,  2.,  8., 18.], grad_fn=<MulBackward0>)

In [11]:
y.sum().backward()  # 等价于 y.backward(torch.ones(len(x)))
x.grad

tensor([ 0.,  4.,  8., 12.])

## 3. `with torch.no_grad()` 

在 PyTorch 中，如果一个张量的 `requires_grad` 参数设为 `True`。则所有依赖它的张量的 `requires_grad` 参数将被设置为 `True`

但在 `with torch.no_grad()` 块中的张量，依赖它的张量的 `requires_grad` 参数将被设为 `False`

参见：https://pytorch.org/docs/stable/generated/torch.no_grad.html

下面是一个例子：

In [12]:
x = torch.tensor([1.], requires_grad=True)
with torch.no_grad():
    y = x * 2
y.requires_grad

False

作为对照：

In [13]:
x = torch.tensor([1.], requires_grad=True)
y = x * 2
y.requires_grad

True